In [1]:
import os
import h5py
import joblib
import logging

import numpy as np
import pandas as pd


from glob import glob

logging.basicConfig(format="[%(levelname)s] %(message)s")

In [2]:
def reduce_memory_usage(df: pd.DataFrame,
                        loglevel: str = None) -> pd.DataFrame:
    """
    Reduce memory usage of dataframe

    Args:
        df (pd.DataFrame): Input dataframe
        loglevel (str, optional): Set log level : "DEBUG", "INFO". Defaults to None.

    Returns:
        pd.DataFrame: Memory optimized dataframe

    
    Reference:
    [1] Reducing memory usage
            https://www.kaggle.com/ragnar123/very-fst-model/notebook
    """
    
    if loglevel:
        logger = logging.getLogger()
        logger.setLevel(loglevel)
        

    prev_memory = df.memory_usage().sum() / 1024**2

    dtypes_numerical = [
        np.uint8, np.uint16, np.uint32, np.uint64,\
        np.int8, np.int16, np.int32, np.int64,\
        np.float16, np.float32, np.float64, np.float128
    ]

    for col in list(df.columns):
        if df[col].dtypes in dtypes_numerical:

            col_min, col_max = df[col].min(), df[col].max()
            col_type = df[col].dtypes
            col_type_name = df[col].dtypes.name
            
            for dtype in dtypes_numerical:
                if 'int' in col_type_name:
                    min_val_supported = np.iinfo(col_type).min
                    max_val_supported = np.iinfo(col_type).max

                elif 'float' in col_type_name:
                    min_val_supported = np.finfo(col_type).min
                    max_val_supported = np.finfo(col_type).max
                else:
                    pass

                if col_min > min_val_supported and col_max < max_val_supported:
                    df[col] = df[col].astype(dtype)
                    
                    if loglevel == "DEBUG":
                        logger.info(f"{col} : changed from {col_type} -> {dtype.__name__}")
                    break

    new_memory = df.memory_usage().sum() / 1024**2

    if loglevel:
        logger.info(f"Previous memory usage {prev_memory:.2f} MB")
        logger.info(f"Optimized memory usage {new_memory:.2f} MB")
    print(f"Percentage reduction in memory {100*(prev_memory-new_memory)/prev_memory:.2f}%")

    return df

In [3]:
def load_data(data_root: str) -> dict:
    """
    Load data with reduced memory usage

    Args:
        data_root (str): Root directory for data files

    Returns:
        data (dict): dictionary with loaded dataframes
        
    Reference:
        [1] Dataframe dtypes
            https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dtypes.html
        
    """

    # dtypes for reducing memory usage

    CALENDAR_DTYPES = {
        "event_name_1": "category",
        "event_name_2": "category",
        "event_type_1": "category",
        "event_type_2": "category",
        "weekday": "category",
    }

    SALESTRAIN_DTYPES = {
        "id": "category",
        "item_id": "category",
        "dept_id": "category",
        "cat_id": "category",
        "store_id": "category",
        "state_id": "category"
    }

    PRICE_DTYPES = {
        "store_id": "category",
        "item_id": "category",
    }

    # load csv and reduce memory usage

    df_cal = pd.read_csv(data_root + "calendar.csv", parse_dates = ['date'])
    df_cal = reduce_memory_usage(df_cal, None)
    df_cal = df_cal.astype(CALENDAR_DTYPES)
    df_cal = df_cal.rename(columns={'d':'day'})

    df_slt = pd.read_csv(data_root + "sales_train_validation.csv")
    df_slt = reduce_memory_usage(df_slt, None)
    df_slt = df_slt.astype(SALESTRAIN_DTYPES)

    df_slp = pd.read_csv(data_root + "sell_prices.csv")
    df_slp = reduce_memory_usage(df_slp, None)
    df_slp = df_slp.astype(PRICE_DTYPES)

    df_sub = pd.read_csv(data_root + "sample_submission.csv")
    df_sub = reduce_memory_usage(df_sub, None)

    data = {
        "calendar": df_cal,
        "sales_train": df_slt,
        "sell_prices": df_slp,
        "submission": df_sub
    }

    return data

In [4]:
if not os.path.exists("../data/processed/data_memopt.bin"):
    data_memopt = load_data("../data/extracted/")
    joblib.dump(data_memopt, filename="../data/processed/data_memopt.bin")
else:
    data_memopt = joblib.load(filename="../data/processed/data_memopt.bin")

In [5]:
def unpivot_data(data: dict) -> dict:
    """
    Unpivot the data for sales_train_validation.csv, submissions.csv

    Args:
        data (str): dictionary with loaded dataframes

    Returns:
        data (dict): dictionary with unpivoted dataframes
    
    Reference
        [1] Melt operation
            https://www.kaggle.com/beezus666/end-to-end-data-wrangling-simple-random-forest
    """
    df_cal = data['calendar']
    df_slt = data['sales_train']
    df_slp = data['sell_prices']
    df_sub = data['submission']

    id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']

    # unpivot sales_train_validation.csv
    df_slt_melted = pd.melt(
        df_slt,
        id_vars=id_vars,
        var_name='day',
        value_name='sales',
    )

    # drop redundant columns from sales_train_validation.csv
    df_slt_melted.drop(
        ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],
        axis=1,
        inplace=True)
    
    # unpivot submissions.csv
    df_sub_melted = pd.melt(
        df_sub,
        id_vars = ['id'],
        value_vars = df_sub.drop(['id'], axis=1).columns,
        var_name = 'day',
        value_name='sales'
    )
    
    df_sub_melted['day'] = df_sub_melted['day'].str.replace('F', '')
    
    # after 1913rd day we need to forecast
    df_sub_melted['day'] = pd.to_numeric(df_sub_melted['day'], errors='coerce')
    df_sub_melted['day'] += 1913 
    df_sub_melted = df_sub_melted.applymap(str)
    df_sub_melted['day'] = 'd_'+ df_sub_melted['day'].astype(str)

    data_melted = {
        "calendar": df_cal,
        "sales_train_melted": df_slt_melted,
        "sell_prices": df_slp,
        "submission_melted": df_sub_melted
    }

    return data_melted

In [6]:
if not os.path.exists("../data/processed/data_melted.bin"):
    data_melted = unpivot_data(data_memopt)
    joblib.dump(data_melted, filename="../data/processed/data_melted.bin")
else:
    data_melted = joblib.load(filename="../data/processed/data_melted.bin")
    del data_memopt

In [7]:
def prepare_data(data: dict, start_day: int = 1000, train_split=.20) -> dict:
    """
    Merge data and prepare train and test sets from unpivoted data

    Args:
        data (str): dictionary with loaded dataframes

    Returns:
        data (dict): dictionary with unpivoted dataframes
    """

    df_cal = data['calendar']
    df_slt = data['sales_train_melted']
    df_slp = data['sell_prices']
    df_sub = data['submission_melted']

    # getting train test split indices
    start_day = 'd_' + str(start_day)
    ix_start = train_ix_start = df_slt[df_slt['day'] == start_day].index[0]
    ix_end = df_slt.index[-1]
    total_samples = ix_end - ix_start + 1
    train_ix_end = train_ix_start + int(total_samples * train_split)

    # train test split
    df_slt['id'] = df_slt['id'].str.replace('_validation', '')
    df_train = df_slt.iloc[train_ix_start:train_ix_end]
    df_test = df_slt.iloc[train_ix_end:]

    # merge with calendar days
    df_train = df_train.merge(right=df_cal,
                              left_on='day',
                              right_on='day',
                              how='left')
    df_test = df_test.merge(right=df_cal,
                            left_on='day',
                            right_on='day',
                            how='left')

    # merging with respective prices
    df_slp['id'] = df_slp['item_id'].astype(str) + '_' + df_slp['store_id'].astype(str)
    df_slp.drop(['item_id', 'store_id'], axis=1, inplace=True)

    # preparing submissions df
    df_sub['id'] = df_sub['id'].str.replace('_evaluation', '')
    df_sub['id'] = df_sub['id'].str.replace('_validation', '')

    df_train = pd.merge(left=df_train,
                        right=df_slp,
                        left_on=['id', 'wm_yr_wk'],
                        right_on=['id', 'wm_yr_wk'])
    df_test = pd.merge(left=df_test,
                       right=df_slp,
                       left_on=['id', 'wm_yr_wk'],
                       right_on=['id', 'wm_yr_wk'])
    
    df_sub = df_sub.merge(df_cal, left_on='day', right_on='day', how='left')

    data_prep = {"train": df_train, "test": df_test, "submission": df_sub}

    return data_prep

In [8]:
if not os.path.exists("../data/processed/data_prep.bin"):
    data_prep = prepare_data(data_melted, start_day=1800, train_split=0.80)
    joblib.dump(data_prep, filename="../data/processed/data_prep.bin")
else:
    data_prep = joblib.load(filename="../data/processed/data_prep.bin")
    del data_melted

In [9]:
data_prep['train'].shape, data_prep['test'].shape, data_prep['submission'].shape

((4417232, 17), (717036, 17), (1707440, 16))

In [10]:
data_prep['train'].columns

Index(['id', 'day', 'sales', 'date', 'wm_yr_wk', 'weekday', 'wday', 'month',
       'year', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2',
       'snap_CA', 'snap_TX', 'snap_WI', 'sell_price'],
      dtype='object')

In [11]:
data_prep['test'].columns

Index(['id', 'day', 'sales', 'date', 'wm_yr_wk', 'weekday', 'wday', 'month',
       'year', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2',
       'snap_CA', 'snap_TX', 'snap_WI', 'sell_price'],
      dtype='object')

In [12]:
data_prep['submission'].columns

Index(['id', 'day', 'sales', 'date', 'wm_yr_wk', 'weekday', 'wday', 'month',
       'year', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2',
       'snap_CA', 'snap_TX', 'snap_WI'],
      dtype='object')

In [13]:
df_train = data_prep['train']
df_test = data_prep['test']
df_submission = data_prep['submission']

# check if any missing values are present
df_train.shape, df_train.count(), df_test.shape, df_test.count(), df_submission.shape, df_submission.count()

((4417232, 17),
 id              4417232
 day             4417232
 sales           4417232
 date            4417232
 wm_yr_wk        4417232
 weekday         4417232
 wday            4417232
 month           4417232
 year            4417232
 event_name_1     512530
 event_type_1     512530
 event_name_2          0
 event_type_2          0
 snap_CA         4417232
 snap_TX         4417232
 snap_WI         4417232
 sell_price      4417232
 dtype: int64,
 (717036, 17),
 id              717036
 day             717036
 sales           717036
 date            717036
 wm_yr_wk        717036
 weekday         717036
 wday            717036
 month           717036
 year            717036
 event_name_1         0
 event_type_1         0
 event_name_2         0
 event_type_2         0
 snap_CA         717036
 snap_TX         717036
 snap_WI         717036
 sell_price      717036
 dtype: int64,
 (1707440, 16),
 id              1707440
 day             1707440
 sales           1707440
 date          